In [5]:
import random
import math

def _init_(self, amostra [], taxa_aprendizado=0.01, saida):
    self.amostras = amostra
    self.saidas = saidas
    self.taxa_aprendizado = taxa_aprendizado
    self.pesos = []
    
def sigmoide(resultado):
   return math.atanh(resultado)

def funcao_ativacao_1(pesos[],taxa_aprendizado_amostra[]):
     

SyntaxError: invalid syntax (<ipython-input-5-29a6348a6784>, line 4)